In [2]:
pip install plotly


Note: you may need to restart the kernel to use updated packages.


In [1]:
import plotly.graph_objects as go
import numpy as np

In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from matplotlib.colors import LinearSegmentedColormap

TO MAKE A VIDEO

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.interpolate as interp
import cv2
import os
from matplotlib.colors import LinearSegmentedColormap

# Data
x = np.array([0, 20, 40, 60, 80, 100, 0, 20, 40, 60, 80, 100, 0, 20, 40, 60, 80, 100])
y = np.array([100, 100, 100, 100, 100, 100, 200, 200, 200, 200, 200, 200, 500, 500, 500, 500, 500, 500])
z = np.array([100, 117.1015964, 105.9850453, 106.7559845, 113.4566645, 114.9838016, 112.7272825, 100, 111.0969213, 110.1921879, 109.8122454, 103.8197951, 100, 78.15530428, 98.9518409, 85.50501254, 89.64694509, 88.6288059])

# Reshape for meshgrid
num_rows = 3  # Number of rows
num_cols = 6  # Number of columns
x_reshaped = x.reshape(num_rows, num_cols)
y_reshaped = y.reshape(num_rows, num_cols)
z_reshaped = z.reshape(num_rows, num_cols)

# Create a finer grid
x_fine = np.linspace(x.min(), x.max(), 100)
y_fine = np.linspace(y.min(), y.max(), 100)
x_fine_grid, y_fine_grid = np.meshgrid(x_fine, y_fine)

# Interpolate z values on the finer grid
interp_func = interp.interp2d(x, y, z, kind='cubic')
z_fine_grid = interp_func(x_fine, y_fine)

# Custom colormap from 40 (blue) to 100 (white) to 160 (red)
colors = [(0, 0, 1), (1, 1, 1), (1, 0, 0)]  # Blue to white to red
n_bins = [40, 100, 160]  # Boundaries
cmap_name = 'custom_cmap'
custom_cmap = LinearSegmentedColormap.from_list(cmap_name, list(zip(np.linspace(0, 1, len(colors)), colors)))

# Directory to save frames
frames_dir = 'frames'
os.makedirs(frames_dir, exist_ok=True)

# Generate frames
num_frames = 120
for i in range(num_frames):
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')

    # Plot surface with custom colormap
    surf = ax.plot_surface(x_fine_grid, y_fine_grid, z_fine_grid, cmap=custom_cmap, edgecolor='none')
    surf.set_clim(40, 160)  # Set color limits directly on the surface plot

    # Add color bar with specific range
    color_bar = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5, pad=0.1)
    color_bar.set_label('MFR (Hz)', fontsize=12)
    color_bar.ax.tick_params(labelsize=10)
    color_bar.outline.set_visible(False)  # Remove black edges of the color bar

    # Rotate the view
    ax.view_init(elev=30, azim=i * 360 / num_frames)

    # Set axis labels and ticks
    ax.set_xlabel('LP (mW)', fontsize=12)
    ax.set_ylabel('Laser pulse (msec)', fontsize=12)
    ax.set_zlabel('MFR (Hz)', fontsize=12)
    ax.tick_params(axis='x', labelsize=10)  # X axis tick labels
    ax.tick_params(axis='y', labelsize=10)  # Y axis tick labels
    ax.tick_params(axis='z', labelsize=10)  # Z axis tick labels

    # Set axis ranges
    ax.set_xlim(x.min(), x.max())
    ax.set_ylim(y.min(), y.max())
    ax.set_zlim(40, 160)

    # Set title
    ax.set_title('Control', loc='center', fontsize=15, pad=20)

    # Save frame
    frame_path = os.path.join(frames_dir, f'frame_{i:03d}.png')
    plt.savefig(frame_path, dpi=300)
    plt.close(fig)

# Save the final image with the title and color bar
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(x_fine_grid, y_fine_grid, z_fine_grid, cmap=custom_cmap, edgecolor='none')
surf.set_clim(40, 160)  # Set color limits directly on the surface plot
color_bar = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5, pad=0.1)
color_bar.set_label('MFR (Hz)', fontsize=12)
color_bar.ax.tick_params(labelsize=10)
color_bar.outline.set_visible(False)  # Remove black edges of the color bar
ax.view_init(elev=30, azim=360)  # Final view angle

# Set axis labels and ticks
ax.set_xlabel('LP (mW)', fontsize=12)
ax.set_ylabel('Laser pulse (msec)', fontsize=12)
ax.set_zlabel('MFR (Hz)', fontsize=12)
ax.tick_params(axis='x', labelsize=10)  # X axis tick labels
ax.tick_params(axis='y', labelsize=10)  # Y axis tick labels
ax.tick_params(axis='z', labelsize=10)  # Z axis tick labels

# Set axis ranges
ax.set_xlim(x.min(), x.max())
ax.set_ylim(y.min(), y.max())
ax.set_zlim(40, 160)

# Set title
ax.set_title('3D Surface Plot', loc='center', fontsize=15, pad=20)

# Save the final frame
final_frame_path = '3d_surface_plot_final.png'
plt.savefig(final_frame_path, dpi=300)
plt.close(fig)

# Video settings
video_path = '3d_surface_plot_video.mp4'
fps = 10  # Frames per second
frame_size = (1280, 720)  # Width and height of the video

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(video_path, fourcc, fps, frame_size)

# Read frames and write to video
for i in range(num_frames):
    frame_path = os.path.join(frames_dir, f'frame_{i:03d}.png')
    frame = cv2.imread(frame_path)
    
    # Resize frame to match the frame_size
    frame = cv2.resize(frame, frame_size)
    
    video_writer.write(frame)

# Release the video writer
video_writer.release()
print(f'Video saved to {video_path}')
print(f'Final image saved as {final_frame_path}')

C:\Users\ane\AppData\Local\Temp\ipykernel_15992\3476864242.py:27: DeprecationWarning: `interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.14.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://scipy.github.io/devdocs/notebooks/interp_transition_guide.html`

  interp_func = interp.interp2d(x, y, z, kind='cubic')
c:\Users\ane\anaconda3\envs\nanoneuro\lib\site-packages\scipy\interpolate\_interpolate.py:286: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too small. (fp>s)
	kx,ky=3,3 nx,ny=9,8 m=18 fp=104.307351 s=0.000000
  self.tck = _fitpack_py.bisplrep(x, y, z, kx=kx, ky=

TO MAKE THE CONTOUR PLOT

In [20]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from matplotlib.colors import LinearSegmentedColormap

# Data arrays
x = np.array([0, 4.1, 8.2, 12.2, 16.3, 20.4, 0, 4.1, 8.2, 12.2, 16.3, 20.4, 0, 4.1, 8.2, 12.2, 16.3, 20.4])
y = np.array([100, 100, 100, 100, 100, 100, 200, 200, 200, 200, 200, 200, 500, 500, 500, 500, 500, 500])
z = np.array([100, 117.1015964, 105.9850453, 106.7559845, 113.4566645, 114.9838016, 112.7272825, 100, 111.0969213, 110.1921879, 109.8122454, 103.8197951, 100, 78.15530428, 98.9518409, 85.50501254, 89.64694509, 88.6288059])

# Create a finer grid
x_fine = np.linspace(x.min(), x.max(), 100)
y_fine = np.linspace(y.min(), y.max(), 100)
x_fine_grid, y_fine_grid = np.meshgrid(x_fine, y_fine)

# Interpolate z values on the finer grid
z_fine_grid = griddata((x, y), z, (x_fine_grid, y_fine_grid), method='cubic')

# Custom colormap from 40 (blue) to 100 (white) to 160 (red)
colors = [(0, 0, 1), (1, 1, 1), (1, 0, 0)]  # Blue to white to red
cmap_name = 'custom_cmap'
custom_cmap = LinearSegmentedColormap.from_list(cmap_name, list(zip(np.linspace(0, 1, len(colors)), colors)))

# Create the contour plot with square aspect ratio
fig, ax = plt.subplots(figsize=(10, 8))

# Create filled contour plot with custom colormap
contourf = ax.contourf(x_fine_grid, y_fine_grid, z_fine_grid, levels=100, cmap=custom_cmap, vmin=-50, vmax=250)

# Add contour lines
contour = ax.contour(x_fine_grid, y_fine_grid, z_fine_grid, levels=10, colors='black', linewidths=0.5)
ax.clabel(contour, fmt='%1.0f', inline=True, fontsize=12)

# Set axis labels and ticks
ax.set_xlabel('Laser power density (W cm$^{-2}$)', fontsize=22, labelpad=20)  # Increase labelpad to separate the label from the plot
ax.set_ylabel('Laser pulse (msec)', fontsize=22, labelpad=20)  # Increase labelpad to separate the label from the plot
ax.tick_params(axis='x', labelsize=18)  # Increase X axis tick labels
ax.tick_params(axis='y', labelsize=18)  # Increase Y axis tick labels
ax.set_xticks(np.linspace(x.min(), x.max(), num=6))  # X axis ticks
ax.set_yticks(np.linspace(y.min(), y.max(), num=5))  # Y axis ticks

# Set axis ranges and aspect ratio to be equal
ax.set_xlim(x.min(), x.max())
ax.set_ylim(y.min(), y.max())


# Set title closer to the plot
plt.title('Control', fontsize=26, pad=20, loc='center')  # Increase title font size and pad

# Add colorbar with an adjusted position to avoid affecting the plot's aspect ratio
cbar = plt.colorbar(contourf, ax=ax, extend='both', shrink=0.8, aspect=10, pad=0.02)
cbar.set_label('Value', fontsize=18)
cbar.set_ticks(np.linspace(40, 240, num=11))  # Set colorbar ticks from 40 to 240
cbar.ax.tick_params(labelsize=14)

# Adjust layout to make sure the plot is square
fig.tight_layout(rect=[0, 0, 0.9, 1])  # Adjust the layout to fit colorbar without affecting the plot's aspect ratio

# Save the image with the top-down perspective
top_view_path = 'CONTROL__contour_plot_top_view.png'
plt.savefig(top_view_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.close(fig)


In [30]:
# linker 5uM

# Data
x = np.array([0, 4.1, 8.2, 12.2, 16.3, 20.4, 0, 4.1, 8.2, 12.2, 16.3, 20.4, 0, 4.1, 8.2, 12.2, 16.3, 20.4])
y = np.array([100, 100, 100, 100, 100, 100, 200, 200, 200, 200, 200, 200, 500, 500, 500, 500, 500, 500])
z = np.array([100,
119.1569058,
133.6251217,
117.8823983,
123.3885175,
118.7245358,
100,
105.7582438,
86.82024,
95.91027813,
93.70215188,
76.52680188,
100,
87.14547385,
91.61229615,
101.7325985,
98.64316385,
98.48603462
])

# Create a finer grid
x_fine = np.linspace(x.min(), x.max(), 100)
y_fine = np.linspace(y.min(), y.max(), 100)
x_fine_grid, y_fine_grid = np.meshgrid(x_fine, y_fine)

# Interpolate z values on the finer grid
z_fine_grid = griddata((x, y), z, (x_fine_grid, y_fine_grid), method='cubic')

# Custom colormap from 40 (blue) to 100 (white) to 160 (red)
colors = [(0, 0, 1), (1, 1, 1), (1, 0, 0)]  # Blue to white to red
cmap_name = 'custom_cmap'
custom_cmap = LinearSegmentedColormap.from_list(cmap_name, list(zip(np.linspace(0, 1, len(colors)), colors)))

# Create the contour plot with square aspect ratio
fig, ax = plt.subplots(figsize=(10, 8))

# Create filled contour plot with custom colormap
contourf = ax.contourf(x_fine_grid, y_fine_grid, z_fine_grid, levels=100, cmap=custom_cmap, vmin=-50, vmax=250)

# Add contour lines
contour = ax.contour(x_fine_grid, y_fine_grid, z_fine_grid, levels=10, colors='black', linewidths=0.5)
ax.clabel(contour, fmt='%1.0f', inline=True, fontsize=12)

# Set axis labels and ticks
ax.set_xlabel('Laser power density (W cm$^{-2}$)', fontsize=22, labelpad=20)  # Increase labelpad to separate the label from the plot
ax.set_ylabel('Laser pulse (msec)', fontsize=22, labelpad=20)  # Increase labelpad to separate the label from the plot
ax.tick_params(axis='x', labelsize=18)  # Increase X axis tick labels
ax.tick_params(axis='y', labelsize=18)  # Increase Y axis tick labels
ax.set_xticks(np.linspace(x.min(), x.max(), num=6))  # X axis ticks
ax.set_yticks(np.linspace(y.min(), y.max(), num=5))  # Y axis ticks

# Set axis ranges and aspect ratio to be equal
ax.set_xlim(x.min(), x.max())
ax.set_ylim(y.min(), y.max())


# Set title closer to the plot
plt.title('5uM linker', fontsize=26, pad=20, loc='center')  # Increase title font size and pad

# Add colorbar with an adjusted position to avoid affecting the plot's aspect ratio
cbar = plt.colorbar(contourf, ax=ax, extend='both', shrink=0.8, aspect=10, pad=0.02)
cbar.set_label('Value', fontsize=18)
cbar.set_ticks(np.linspace(40, 240, num=11))  # Set colorbar ticks from 40 to 240
cbar.ax.tick_params(labelsize=14)

# Save the image with the top-down perspective
top_view_path = '5uM linker__contour_plot_top_view.png'
plt.savefig(top_view_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.close(fig)

In [31]:
# BPs 0.002nM 

# Data
x = np.array([0, 4.1, 8.2, 12.2, 16.3, 20.4, 0, 4.1, 8.2, 12.2, 16.3, 20.4, 0, 4.1, 8.2, 12.2, 16.3, 20.4])
y = np.array([100, 100, 100, 100, 100, 100, 200, 200, 200, 200, 200, 200, 500, 500, 500, 500, 500, 500])
z = np.array([100,
110.6695162,
112.0161562,
118.1447546,
113.4844323,
133.2842623,
100,
105.2399591,
101.3914782,
99.67101545,
104.3631918,
108.7312709,
100,
91.6238125,
92.91018667,
105.0949133,
111.0235475,
112.5350483
])

# Create a finer grid
x_fine = np.linspace(x.min(), x.max(), 100)
y_fine = np.linspace(y.min(), y.max(), 100)
x_fine_grid, y_fine_grid = np.meshgrid(x_fine, y_fine)

# Interpolate z values on the finer grid
z_fine_grid = griddata((x, y), z, (x_fine_grid, y_fine_grid), method='cubic')

# Custom colormap from 40 (blue) to 100 (white) to 160 (red)
colors = [(0, 0, 1), (1, 1, 1), (1, 0, 0)]  # Blue to white to red
cmap_name = 'custom_cmap'
custom_cmap = LinearSegmentedColormap.from_list(cmap_name, list(zip(np.linspace(0, 1, len(colors)), colors)))

# Create the contour plot with square aspect ratio
fig, ax = plt.subplots(figsize=(10, 8))

# Create filled contour plot with custom colormap
contourf = ax.contourf(x_fine_grid, y_fine_grid, z_fine_grid, levels=100, cmap=custom_cmap, vmin=-50, vmax=250)

# Add contour lines
contour = ax.contour(x_fine_grid, y_fine_grid, z_fine_grid, levels=10, colors='black', linewidths=0.5)
ax.clabel(contour, fmt='%1.0f', inline=True, fontsize=12)

# Set axis labels and ticks
ax.set_xlabel('Laser power density (W cm$^{-2}$)', fontsize=22, labelpad=20)  # Increase labelpad to separate the label from the plot
ax.set_ylabel('Laser pulse (msec)', fontsize=22, labelpad=20)  # Increase labelpad to separate the label from the plot
ax.tick_params(axis='x', labelsize=18)  # Increase X axis tick labels
ax.tick_params(axis='y', labelsize=18)  # Increase Y axis tick labels
ax.set_xticks(np.linspace(x.min(), x.max(), num=6))  # X axis ticks
ax.set_yticks(np.linspace(y.min(), y.max(), num=5))  # Y axis ticks

# Set axis ranges and aspect ratio to be equal
ax.set_xlim(x.min(), x.max())
ax.set_ylim(y.min(), y.max())


# Set title closer to the plot
plt.title('0.002nM BPs', fontsize=26, pad=20, loc='center')  # Increase title font size and pad

# Add colorbar with an adjusted position to avoid affecting the plot's aspect ratio
cbar = plt.colorbar(contourf, ax=ax, extend='both', shrink=0.8, aspect=10, pad=0.02)
cbar.set_label('Value', fontsize=18)
cbar.set_ticks(np.linspace(40, 240, num=11))  # Set colorbar ticks from 40 to 240
cbar.ax.tick_params(labelsize=14)

# Save the image with the top-down perspective
top_view_path = '0.002nM BPs__contour_plot_top_view.png'
plt.savefig(top_view_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.close(fig)

In [32]:
# BPs 0.02 nM

# Data
x = np.array([0, 4.1, 8.2, 12.2, 16.3, 20.4, 0, 4.1, 8.2, 12.2, 16.3, 20.4, 0, 4.1, 8.2, 12.2, 16.3, 20.4])
y = np.array([100, 100, 100, 100, 100, 100, 200, 200, 200, 200, 200, 200, 500, 500, 500, 500, 500, 500])
z = np.array([100,
95.61503167,
96.42577417,
91.6268075,
92.08605667,
84.703685,
100,
90.134582,
103.0334027,
98.124806,
97.96379667,
102.1050773,
100,
98.19122083,
107.2928525,
118.2329,
132.5045225,
150.0132017
])

# Create a finer grid
x_fine = np.linspace(x.min(), x.max(), 100)
y_fine = np.linspace(y.min(), y.max(), 100)
x_fine_grid, y_fine_grid = np.meshgrid(x_fine, y_fine)

# Interpolate z values on the finer grid
z_fine_grid = griddata((x, y), z, (x_fine_grid, y_fine_grid), method='cubic')

# Custom colormap from 40 (blue) to 100 (white) to 160 (red)
colors = [(0, 0, 1), (1, 1, 1), (1, 0, 0)]  # Blue to white to red
cmap_name = 'custom_cmap'
custom_cmap = LinearSegmentedColormap.from_list(cmap_name, list(zip(np.linspace(0, 1, len(colors)), colors)))

# Create the contour plot with square aspect ratio
fig, ax = plt.subplots(figsize=(10, 8))

# Create filled contour plot with custom colormap
contourf = ax.contourf(x_fine_grid, y_fine_grid, z_fine_grid, levels=100, cmap=custom_cmap, vmin=-50, vmax=250)

# Add contour lines
contour = ax.contour(x_fine_grid, y_fine_grid, z_fine_grid, levels=10, colors='black', linewidths=0.5)
ax.clabel(contour, fmt='%1.0f', inline=True, fontsize=12)

# Set axis labels and ticks
ax.set_xlabel('Laser power density (W cm$^{-2}$)', fontsize=22, labelpad=20)  # Increase labelpad to separate the label from the plot
ax.set_ylabel('Laser pulse (msec)', fontsize=22, labelpad=20)  # Increase labelpad to separate the label from the plot
ax.tick_params(axis='x', labelsize=18)  # Increase X axis tick labels
ax.tick_params(axis='y', labelsize=18)  # Increase Y axis tick labels
ax.set_xticks(np.linspace(x.min(), x.max(), num=6))  # X axis ticks
ax.set_yticks(np.linspace(y.min(), y.max(), num=5))  # Y axis ticks

# Set axis ranges and aspect ratio to be equal
ax.set_xlim(x.min(), x.max())
ax.set_ylim(y.min(), y.max())


# Set title closer to the plot
plt.title('0.02nM BPs', fontsize=26, pad=20, loc='center')  # Increase title font size and pad

# Add colorbar with an adjusted position to avoid affecting the plot's aspect ratio
cbar = plt.colorbar(contourf, ax=ax, extend='both', shrink=0.8, aspect=10, pad=0.02)
cbar.set_label('Value', fontsize=18)
cbar.set_ticks(np.linspace(40, 240, num=11))  # Set colorbar ticks from 40 to 240
cbar.ax.tick_params(labelsize=14)

# Save the image with the top-down perspective
top_view_path = '0.02nM BPs__contour_plot_top_view.png'
plt.savefig(top_view_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.close(fig)

In [33]:
# BPs 0.05 nM

# Data
x = np.array([0, 4.1, 8.2, 12.2, 16.3, 20.4, 0, 4.1, 8.2, 12.2, 16.3, 20.4, 0, 4.1, 8.2, 12.2, 16.3, 20.4])
y = np.array([100, 100, 100, 100, 100, 100, 200, 200, 200, 200, 200, 200, 500, 500, 500, 500, 500, 500])
z = np.array([100,
99.43486071,
118.7686529,
119.9866064,
129.4073936,
143.1732614,
100,
103.226383,
137.55223,
124.738149,
138.364314,
146.96101,
100,
98.62262833,
106.3522133,
116.8326375,
135.535725,
138.5444842
])

# Create a finer grid
x_fine = np.linspace(x.min(), x.max(), 100)
y_fine = np.linspace(y.min(), y.max(), 100)
x_fine_grid, y_fine_grid = np.meshgrid(x_fine, y_fine)

# Interpolate z values on the finer grid
z_fine_grid = griddata((x, y), z, (x_fine_grid, y_fine_grid), method='cubic')

# Custom colormap from 40 (blue) to 100 (white) to 160 (red)
colors = [(0, 0, 1), (1, 1, 1), (1, 0, 0)]  # Blue to white to red
cmap_name = 'custom_cmap'
custom_cmap = LinearSegmentedColormap.from_list(cmap_name, list(zip(np.linspace(0, 1, len(colors)), colors)))

# Create the contour plot with square aspect ratio
fig, ax = plt.subplots(figsize=(10, 8))

# Create filled contour plot with custom colormap
contourf = ax.contourf(x_fine_grid, y_fine_grid, z_fine_grid, levels=100, cmap=custom_cmap, vmin=-50, vmax=250)

# Add contour lines
contour = ax.contour(x_fine_grid, y_fine_grid, z_fine_grid, levels=10, colors='black', linewidths=0.5)
ax.clabel(contour, fmt='%1.0f', inline=True, fontsize=12)

# Set axis labels and ticks
ax.set_xlabel('Laser power density (W cm$^{-2}$)', fontsize=22, labelpad=20)  # Increase labelpad to separate the label from the plot
ax.set_ylabel('Laser pulse (msec)', fontsize=22, labelpad=20)  # Increase labelpad to separate the label from the plot
ax.tick_params(axis='x', labelsize=18)  # Increase X axis tick labels
ax.tick_params(axis='y', labelsize=18)  # Increase Y axis tick labels
ax.set_xticks(np.linspace(x.min(), x.max(), num=6))  # X axis ticks
ax.set_yticks(np.linspace(y.min(), y.max(), num=5))  # Y axis ticks

# Set axis ranges and aspect ratio to be equal
ax.set_xlim(x.min(), x.max())
ax.set_ylim(y.min(), y.max())


# Set title closer to the plot
plt.title('0.05nM BPs', fontsize=26, pad=20, loc='center')  # Increase title font size and pad

# Add colorbar with an adjusted position to avoid affecting the plot's aspect ratio
cbar = plt.colorbar(contourf, ax=ax, extend='both', shrink=0.8, aspect=10, pad=0.02)
cbar.set_label('Value', fontsize=18)
cbar.set_ticks(np.linspace(40, 240, num=11))  # Set colorbar ticks from 40 to 240
cbar.ax.tick_params(labelsize=14)

# Save the image with the top-down perspective
top_view_path = '0.05nM BPs__contour_plot_top_view.png'
plt.savefig(top_view_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.close(fig)

In [40]:
# 0.5uM linker + 0.002 nM BPs

# Data
x = np.array([0, 4.1, 8.2, 12.2, 16.3, 20.4, 0, 8.2, 12.2, 16.3, 20.4, 0, 4.1, 8.2, 12.2, 16.3, 20.4])
y = np.array([100, 100, 100, 100, 100, 100, 200, 200, 200, 200, 200, 500, 500, 500, 500, 500, 500])
z = np.array([100,
117.1015929,
105.9850336,
106.7559843,
113.4566679,
114.9838071,
100,
100.62949,
94.1127725,
129.942675,
134.050425,
100,
72.70507615,
99.08377231,
85.45436231,
79.92556077,
118.0387646
])

# Create a finer grid
x_fine = np.linspace(x.min(), x.max(), 100)
y_fine = np.linspace(y.min(), y.max(), 100)
x_fine_grid, y_fine_grid = np.meshgrid(x_fine, y_fine)

# Interpolate z values on the finer grid
z_fine_grid = griddata((x, y), z, (x_fine_grid, y_fine_grid), method='cubic')

# Custom colormap from 40 (blue) to 100 (white) to 160 (red)
colors = [(0, 0, 1), (1, 1, 1), (1, 0, 0)]  # Blue to white to red
cmap_name = 'custom_cmap'
custom_cmap = LinearSegmentedColormap.from_list(cmap_name, list(zip(np.linspace(0, 1, len(colors)), colors)))

# Create the contour plot with square aspect ratio
fig, ax = plt.subplots(figsize=(10, 8))

# Create filled contour plot with custom colormap
contourf = ax.contourf(x_fine_grid, y_fine_grid, z_fine_grid, levels=100, cmap=custom_cmap, vmin=-50, vmax=250)

# Add contour lines
contour = ax.contour(x_fine_grid, y_fine_grid, z_fine_grid, levels=10, colors='black', linewidths=0.5)
ax.clabel(contour, fmt='%1.0f', inline=True, fontsize=12)

# Set axis labels and ticks
ax.set_xlabel('Laser power density (W cm$^{-2}$)', fontsize=22, labelpad=20)  # Increase labelpad to separate the label from the plot
ax.set_ylabel('Laser pulse (msec)', fontsize=22, labelpad=20)  # Increase labelpad to separate the label from the plot
ax.tick_params(axis='x', labelsize=18)  # Increase X axis tick labels
ax.tick_params(axis='y', labelsize=18)  # Increase Y axis tick labels
ax.set_xticks(np.linspace(x.min(), x.max(), num=6))  # X axis ticks
ax.set_yticks(np.linspace(y.min(), y.max(), num=5))  # Y axis ticks

# Set axis ranges and aspect ratio to be equal
ax.set_xlim(x.min(), x.max())
ax.set_ylim(y.min(), y.max())


# Set title closer to the plot
plt.title('0.5uM Linker + 0.002nM BPs', fontsize=26, pad=20, loc='center')  # Increase title font size and pad

# Add colorbar with an adjusted position to avoid affecting the plot's aspect ratio
cbar = plt.colorbar(contourf, ax=ax, extend='both', shrink=0.8, aspect=10, pad=0.02)
cbar.set_label('Value', fontsize=18)
cbar.set_ticks(np.linspace(40, 240, num=11))  # Set colorbar ticks from 40 to 240
cbar.ax.tick_params(labelsize=14)

# Save the image with the top-down perspective
top_view_path = '0.5uM Linker + 0.002nM BPs__contour_plot_top_view.png'
plt.savefig(top_view_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.close(fig)

In [35]:
# 2uM linker + 0.002 nM BPs

# Data
x = np.array([0, 4.1, 8.2, 12.2, 16.3, 20.4, 0, 4.1, 8.2, 12.2, 16.3, 20.4, 0, 4.1, 8.2, 12.2, 16.3, 20.4])
y = np.array([100, 100, 100, 100, 100, 100, 200, 200, 200, 200, 200, 200, 500, 500, 500, 500, 500, 500])
z = np.array([100,
108.7895886,
105.7611107,
109.1451486,
104.1084779,
108.4925564,
100,
73.838264,
96.787168,
83.480464,
145.81782,
116.078776,
100,
114.4212143,
117.42605,
126.7381786,
123.55356,
130.7185786,
])

## Create a finer grid
x_fine = np.linspace(x.min(), x.max(), 100)
y_fine = np.linspace(y.min(), y.max(), 100)
x_fine_grid, y_fine_grid = np.meshgrid(x_fine, y_fine)

# Interpolate z values on the finer grid
z_fine_grid = griddata((x, y), z, (x_fine_grid, y_fine_grid), method='cubic')

# Custom colormap from 40 (blue) to 100 (white) to 160 (red)
colors = [(0, 0, 1), (1, 1, 1), (1, 0, 0)]  # Blue to white to red
cmap_name = 'custom_cmap'
custom_cmap = LinearSegmentedColormap.from_list(cmap_name, list(zip(np.linspace(0, 1, len(colors)), colors)))

# Create the contour plot with square aspect ratio
fig, ax = plt.subplots(figsize=(10, 8))

# Create filled contour plot with custom colormap
contourf = ax.contourf(x_fine_grid, y_fine_grid, z_fine_grid, levels=100, cmap=custom_cmap, vmin=-50, vmax=250)

# Add contour lines
contour = ax.contour(x_fine_grid, y_fine_grid, z_fine_grid, levels=10, colors='black', linewidths=0.5)
ax.clabel(contour, fmt='%1.0f', inline=True, fontsize=12)

# Set axis labels and ticks
ax.set_xlabel('Laser power density (W cm$^{-2}$)', fontsize=22, labelpad=20)  # Increase labelpad to separate the label from the plot
ax.set_ylabel('Laser pulse (msec)', fontsize=22, labelpad=20)  # Increase labelpad to separate the label from the plot
ax.tick_params(axis='x', labelsize=18)  # Increase X axis tick labels
ax.tick_params(axis='y', labelsize=18)  # Increase Y axis tick labels
ax.set_xticks(np.linspace(x.min(), x.max(), num=6))  # X axis ticks
ax.set_yticks(np.linspace(y.min(), y.max(), num=5))  # Y axis ticks

# Set axis ranges and aspect ratio to be equal
ax.set_xlim(x.min(), x.max())
ax.set_ylim(y.min(), y.max())


# Set title closer to the plot
plt.title('2uM Linker + 0.002nM BPs', fontsize=26, pad=20, loc='center')  # Increase title font size and pad

# Add colorbar with an adjusted position to avoid affecting the plot's aspect ratio
cbar = plt.colorbar(contourf, ax=ax, extend='both', shrink=0.8, aspect=10, pad=0.02)
cbar.set_label('Value', fontsize=18)
cbar.set_ticks(np.linspace(40, 240, num=11))  # Set colorbar ticks from 40 to 240
cbar.ax.tick_params(labelsize=14)

# Save the image with the top-down perspective
top_view_path = '2uM Linker + 0.002nM BPs__contour_plot_top_view.png'
plt.savefig(top_view_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.close(fig)

In [39]:
# 5uM linker + 0.002 nM BPs

# Data
x = np.array([0, 4.1, 8.2, 12.2, 16.3, 20.4, 0, 4.1, 8.2, 12.2, 16.3, 0, 4.1, 8.2, 12.2, 16.3, 20.4])
y = np.array([100, 100, 100, 100, 100, 100, 200, 200, 200, 200, 200, 500, 500, 500, 500, 500, 500])
z = np.array([100,
100.3922578,
99.83313667,
109.4287067,
110.6151522,
107.1792444,
100,
85.83160578,
96.05056315,
100.9484365,
87.77511408,
100,
152.4231816,
124.7770594,
118.8104187,
146.7616517,
239.4525931
])

# Create a finer grid
x_fine = np.linspace(x.min(), x.max(), 100)
y_fine = np.linspace(y.min(), y.max(), 100)
x_fine_grid, y_fine_grid = np.meshgrid(x_fine, y_fine)

# Interpolate z values on the finer grid
z_fine_grid = griddata((x, y), z, (x_fine_grid, y_fine_grid), method='cubic')

# Custom colormap from 40 (blue) to 100 (white) to 160 (red)
colors = [(0, 0, 1), (1, 1, 1), (1, 0, 0)]  # Blue to white to red
cmap_name = 'custom_cmap'
custom_cmap = LinearSegmentedColormap.from_list(cmap_name, list(zip(np.linspace(0, 1, len(colors)), colors)))

# Create the contour plot with square aspect ratio
fig, ax = plt.subplots(figsize=(10, 8))

# Create filled contour plot with custom colormap
contourf = ax.contourf(x_fine_grid, y_fine_grid, z_fine_grid, levels=100, cmap=custom_cmap, vmin=-50, vmax=250)

# Add contour lines
contour = ax.contour(x_fine_grid, y_fine_grid, z_fine_grid, levels=10, colors='black', linewidths=0.5)
ax.clabel(contour, fmt='%1.0f', inline=True, fontsize=12)

# Set axis labels and ticks
ax.set_xlabel('Laser power density (W cm$^{-2}$)', fontsize=22, labelpad=20)  # Increase labelpad to separate the label from the plot
ax.set_ylabel('Laser pulse (msec)', fontsize=22, labelpad=20)  # Increase labelpad to separate the label from the plot
ax.tick_params(axis='x', labelsize=18)  # Increase X axis tick labels
ax.tick_params(axis='y', labelsize=18)  # Increase Y axis tick labels
ax.set_xticks(np.linspace(x.min(), x.max(), num=6))  # X axis ticks
ax.set_yticks(np.linspace(y.min(), y.max(), num=5))  # Y axis ticks

# Set axis ranges and aspect ratio to be equal
ax.set_xlim(x.min(), x.max())
ax.set_ylim(y.min(), y.max())


# Set title closer to the plot
plt.title('5uM Linker + 0.002nM BPs', fontsize=26, pad=20, loc='center')  # Increase title font size and pad

# Add colorbar with an adjusted position to avoid affecting the plot's aspect ratio
cbar = plt.colorbar(contourf, ax=ax, extend='both', shrink=0.8, aspect=10, pad=0.02)
cbar.set_label('Value', fontsize=18)
cbar.set_ticks(np.linspace(40, 240, num=11))  # Set colorbar ticks from 40 to 240
cbar.ax.tick_params(labelsize=14)

# Save the image with the top-down perspective
top_view_path = '5uM Linker + 0.002nM BPs__contour_plot_top_view.png'
plt.savefig(top_view_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.close(fig)

In [37]:
# 5uM linker + 0.02 nM BPs

# Data
x = np.array([0, 4.1, 8.2, 12.2, 16.3, 20.4, 0, 4.1, 8.2, 12.2, 16.3, 20.4, 0, 4.1, 8.2, 12.2, 16.3, 20.4])
y = np.array([100, 100, 100, 100, 100, 100, 200, 200, 200, 200, 200, 200, 500, 500, 500, 500, 500, 500])
z = np.array([100,
108.58452,
96.454142,
104.339166,
103.95342,
101.52748,
100,
82.331972,
107.67896,
68.013824,
113.469076,
75.965972,
100,
98.33286909,
110.2056727,
98.72572818,
108.3374382,
104.9167191
])

# Create a finer grid
x_fine = np.linspace(x.min(), x.max(), 100)
y_fine = np.linspace(y.min(), y.max(), 100)
x_fine_grid, y_fine_grid = np.meshgrid(x_fine, y_fine)

# Interpolate z values on the finer grid
z_fine_grid = griddata((x, y), z, (x_fine_grid, y_fine_grid), method='cubic')

# Custom colormap from 40 (blue) to 100 (white) to 160 (red)
colors = [(0, 0, 1), (1, 1, 1), (1, 0, 0)]  # Blue to white to red
cmap_name = 'custom_cmap'
custom_cmap = LinearSegmentedColormap.from_list(cmap_name, list(zip(np.linspace(0, 1, len(colors)), colors)))

# Create the contour plot with square aspect ratio
fig, ax = plt.subplots(figsize=(10, 8))

# Create filled contour plot with custom colormap
contourf = ax.contourf(x_fine_grid, y_fine_grid, z_fine_grid, levels=100, cmap=custom_cmap, vmin=-50, vmax=250)

# Add contour lines
contour = ax.contour(x_fine_grid, y_fine_grid, z_fine_grid, levels=10, colors='black', linewidths=0.5)
ax.clabel(contour, fmt='%1.0f', inline=True, fontsize=12)

# Set axis labels and ticks
ax.set_xlabel('Laser power density (W cm$^{-2}$)', fontsize=22, labelpad=20)  # Increase labelpad to separate the label from the plot
ax.set_ylabel('Laser pulse (msec)', fontsize=22, labelpad=20)  # Increase labelpad to separate the label from the plot
ax.tick_params(axis='x', labelsize=18)  # Increase X axis tick labels
ax.tick_params(axis='y', labelsize=18)  # Increase Y axis tick labels
ax.set_xticks(np.linspace(x.min(), x.max(), num=6))  # X axis ticks
ax.set_yticks(np.linspace(y.min(), y.max(), num=5))  # Y axis ticks

# Set axis ranges and aspect ratio to be equal
ax.set_xlim(x.min(), x.max())
ax.set_ylim(y.min(), y.max())


# Set title closer to the plot
plt.title('5uM Linker + 0.02nM BPs', fontsize=26, pad=20, loc='center')  # Increase title font size and pad

# Add colorbar with an adjusted position to avoid affecting the plot's aspect ratio
cbar = plt.colorbar(contourf, ax=ax, extend='both', shrink=0.8, aspect=10, pad=0.02)
cbar.set_label('Value', fontsize=18)
cbar.set_ticks(np.linspace(40, 240, num=11))  # Set colorbar ticks from 40 to 240
cbar.ax.tick_params(labelsize=14)

# Save the image with the top-down perspective
top_view_path = '5uM Linker + 0.02nM BPs__contour_plot_top_view.png'
plt.savefig(top_view_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.close(fig)

In [2]:
# 5uM linker + 0.05 nM BPs

# Data
x = np.array([0, 4.1, 8.2, 12.2, 16.3, 20.4, 0, 4.1, 8.2, 12.2, 16.3, 20.4, 0, 8.2, 12.2, 16.3, 20.4])
y = np.array([100, 100, 100, 100, 100, 100, 200, 200, 200, 200, 200, 200, 500, 500, 500, 500, 500])
z = np.array([100,
86.23809364,
88.42591455,
106.3455436,
99.52840455,
87.76439818,
100,
110.198704,
120.3104,
147.98892,
98.624144,
178.9371,
100,
136.8006636,
129.4482682,
132.7015764,
152.3456364
])

# Create a finer grid
x_fine = np.linspace(x.min(), x.max(), 100)
y_fine = np.linspace(y.min(), y.max(), 100)
x_fine_grid, y_fine_grid = np.meshgrid(x_fine, y_fine)

# Interpolate z values on the finer grid
z_fine_grid = griddata((x, y), z, (x_fine_grid, y_fine_grid), method='cubic')

# Custom colormap from 40 (blue) to 100 (white) to 160 (red)
colors = [(0, 0, 1), (1, 1, 1), (1, 0, 0)]  # Blue to white to red
cmap_name = 'custom_cmap'
custom_cmap = LinearSegmentedColormap.from_list(cmap_name, list(zip(np.linspace(0, 1, len(colors)), colors)))

# Create the contour plot with square aspect ratio
fig, ax = plt.subplots(figsize=(10, 8))

# Create filled contour plot with custom colormap
contourf = ax.contourf(x_fine_grid, y_fine_grid, z_fine_grid, levels=100, cmap=custom_cmap, vmin=-50, vmax=250)

# Add contour lines
contour = ax.contour(x_fine_grid, y_fine_grid, z_fine_grid, levels=10, colors='black', linewidths=0.5)
ax.clabel(contour, fmt='%1.0f', inline=True, fontsize=12)

# Set axis labels and ticks
ax.set_xlabel('Laser power density (W cm$^{-2}$)', fontsize=22, labelpad=20)  # Increase labelpad to separate the label from the plot
ax.set_ylabel('Laser pulse (msec)', fontsize=22, labelpad=20)  # Increase labelpad to separate the label from the plot
ax.tick_params(axis='x', labelsize=18)  # Increase X axis tick labels
ax.tick_params(axis='y', labelsize=18)  # Increase Y axis tick labels
ax.set_xticks(np.linspace(x.min(), x.max(), num=6))  # X axis ticks
ax.set_yticks(np.linspace(y.min(), y.max(), num=5))  # Y axis ticks

# Set axis ranges and aspect ratio to be equal
ax.set_xlim(x.min(), x.max())
ax.set_ylim(y.min(), y.max())


# Set title closer to the plot
plt.title('5uM Linker + 0.05nM BPs', fontsize=26, pad=20, loc='center')  # Increase title font size and pad

# Add colorbar with an adjusted position to avoid affecting the plot's aspect ratio
cbar = plt.colorbar(contourf, ax=ax, extend='both', shrink=0.8, aspect=10, pad=0.02)
cbar.set_label('Value', fontsize=18)
cbar.set_ticks(np.linspace(40, 240, num=11))  # Set colorbar ticks from 40 to 240
cbar.ax.tick_params(labelsize=14)

# Save the image with the top-down perspective
top_view_path = '5uM Linker + 0.05nM BPs__contour_plot_top_view.png'
plt.savefig(top_view_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.close(fig)